In [1]:
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer
import random
from prompt_templates import create_prompt_subtitles
from transformers import MT5ForConditionalGeneration
import argparse
from datetime import datetime
from models import get_model
from dataloader import get_loader
import torch
from torch import nn
from evaluate import evaluate
import copy
import wandb
from tqdm import tqdm
import os
import json
from torch.nn import CrossEntropyLoss
from transformers import AutoTokenizer
from scheduler import CosineWarmupScheduler
from utils import compute_rank_loss
from torch.cuda.amp import GradScaler


/home/colin/miniconda3/envs/recsys/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%load_ext autoreload
%autoreload 2

In [21]:
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")
model

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

In [22]:
model.base_model.load_state_dict(torch.load('checkpoints/model_0.0001.pth',map_location=torch.device('cpu')))

<All keys matched successfully>

In [5]:
class b:
    dataset = "demo"
    batch_size = 3
    num_workers = 4
    titles = True
    T=4
    datafraction=0.001

b = b()

In [6]:
data_loader = get_loader(b, 'train', tokenizer, debug=True)

In [7]:
data_loader2 = get_loader(b, 'train', tokenizer, debug=False)


In [8]:
out1= next(iter(data_loader2))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/home/colin/miniconda3/envs/recsys/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/colin/

In [9]:
out1.keys()

dict_keys(['pos_input_ids', 'pos_attention_mask', 'neg_input_ids', 'neg_attention_mask', 'pos_labels', 'neg_labels', 'decoder_start'])

In [10]:
out1['pos_input_ids'].shape, out1['pos_attention_mask'].shape, out1['neg_input_ids'].shape, out1['pos_labels'].shape, out1['neg_attention_mask'].shape, out1["decoder_start"].shape

(torch.Size([3, 110]),
 torch.Size([3, 110]),
 torch.Size([3, 110]),
 torch.Size([3, 2]),
 torch.Size([3, 110]),
 torch.Size([3, 5]))

In [11]:
for i in range(3):
    print(tokenizer.decode(out1['pos_input_ids'][i].tolist()))
    print(tokenizer.decode(out1['neg_input_ids'][i].tolist()))  

En bruger har for nylig læst artikler: Midt i skole-fejring: Rykker ud til knivstikkeri, Sender stikpille til dommer: - Han læser aftrykkene forkert, Knivstikkeri i Fælledparken: En anholdt og sigtet, Total ydmygelse: City smadrer Real Madrid og er klar til CL-, vil brugeren læse artiklen - Hvordan skal jeg svare på det?? (ja/nej)</s>
En bruger har for nylig læst artikler: Midt i skole-fejring: Rykker ud til knivstikkeri, Sender stikpille til dommer: - Han læser aftrykkene forkert, Knivstikkeri i Fælledparken: En anholdt og sigtet, Total ydmygelse: City smadrer Real Madrid og er klar til CL-, vil brugeren læse artiklen Lufthavnskaos: Her er dine rettigheder? (ja/nej)</s>
En bruger har for nylig læst artikler: Bortført pige fundet efter seks år: Genkendt i Netflix-serie, Træt af spioner: For mange udnytter det, Så klappede fælden: Techsvindler skal bag tremmer, Derfor kom Kjær ikke ind: Var for langsom, vil brugeren læse artiklen Kom med på fremtidens McDonald's? (ja/nej)</s><pad><pad><

In [12]:
tokenizer.encode("ja / nej"), tokenizer.encode("ja"), tokenizer.encode("nej")

([432, 259, 275, 3810, 1], [432, 1], [3810, 1])

In [13]:
pos_outputs = model(
    input_ids=out1["pos_input_ids"], 
    attention_mask=out1["pos_attention_mask"],
    decoder_input_ids=out1["decoder_start"],
)
neg_outputs = model(
    input_ids=out1["neg_input_ids"], 
    attention_mask=out1["neg_attention_mask"],
    decoder_input_ids=out1["decoder_start"],
)


In [14]:
ja = tokenizer.encode('ja')[0]
nej = tokenizer.encode('nej')[0]

other_logit_idxs = [i for i in range(pos_outputs.logits.shape[2])]
other_logit_idxs.remove(ja)
other_logit_idxs.remove(nej)

print('== pos examples ==')
print('ja:',pos_outputs.logits[:,0,ja])
print('nej:',pos_outputs.logits[:,0,nej])
other_logits_pos = pos_outputs.logits[:,0,other_logit_idxs]
print('all other tokens (mean):',torch.mean(other_logits_pos,dim=-1))
print('all other tokens (std):',torch.std(other_logits_pos,dim=-1))



print('== pos examples ==')
print('ja:',neg_outputs.logits[:,0,ja])
print('nej:',neg_outputs.logits[:,0,nej])
other_logits_neg = neg_outputs.logits[:,0,other_logit_idxs]
print('all other tokens (mean):',torch.mean(other_logits_neg,dim=-1))
print('all other tokens (std):',torch.std(other_logits_neg,dim=-1))

== pos examples ==
ja: tensor([4.2663, 4.7666, 4.8914], grad_fn=<SelectBackward0>)
nej: tensor([5.4130, 5.3510, 4.7950], grad_fn=<SelectBackward0>)
all other tokens (mean): tensor([-22.8444, -22.8865, -22.5977], grad_fn=<MeanBackward1>)
all other tokens (std): tensor([0.4547, 0.4488, 0.4111], grad_fn=<StdBackward0>)
== pos examples ==
ja: tensor([4.0282, 4.5596, 4.9551], grad_fn=<SelectBackward0>)
nej: tensor([5.6186, 5.4770, 5.0416], grad_fn=<SelectBackward0>)
all other tokens (mean): tensor([-22.9179, -22.7150, -22.6757], grad_fn=<MeanBackward1>)
all other tokens (std): tensor([0.4995, 0.4571, 0.4148], grad_fn=<StdBackward0>)


In [15]:
out1["pos_input_ids"].shape

torch.Size([3, 110])

In [23]:
from transformers import AutoTokenizer, AutoModel, utils
from bertviz import model_view
utils.logging.set_verbosity_error()  # Suppress standard warnings

# model_name = "google/mt5-small"  # Find popular HuggingFace models here: https://huggingface.co/models
# input_text = "The cat sat on the mat"  
# model = MT5ForConditionalGeneration.from_pretrained(model_name, output_attentions=True)  # Configure model to return attention values
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# inputs = tokenizer.encode(input_text, return_tensors='pt')  # Tokenize input text
outputs = model(input_ids=out1["pos_input_ids"][0].unsqueeze(0), attention_mask=out1["pos_attention_mask"][0].unsqueeze(0), decoder_input_ids=out1["decoder_start"][0].unsqueeze(0))  # Run model
attention = outputs[-1]  # Retrieve attention from model outputs


In [17]:
outputs[0].shape

torch.Size([1, 5, 250112])

In [24]:
tokens = tokenizer.convert_ids_to_tokens(out1["pos_input_ids"][0])  # Convert input ids to token strings


In [25]:
model_view(attention, tokens)  # Display model view

ValueError: The attention tensor does not have the correct number of dimensions. Make sure you set output_attentions=True when initializing your model.